In [ ]:
import argparse
import collections.abc
import errno
import numpy as np
import os
import pandas as pd
import pathlib
import scipy
import shutil
import sys
import time
import warnings
import SimpleITK as sitk

from datetime import datetime
from glob import glob
from importlib import import_module
from os import listdir
from os.path import dirname, exists, isdir, join
from pynvml import *

import torch
from torch import optim
from torch.autograd import Variable
from torch.backends import cudnn
from torch.nn import DataParallel
from torch.utils.data import Dataset, DataLoader

RootPath = r''

if RootPath not in sys.path: sys.path.insert(0, RootPath)
from configs.config03 import get_cfg_defaults

CodePath = join(RootPath, 'code')
if CodePath not in sys.path: sys.path.insert(1, CodePath)
from models import res18_se

display(sys.path)

In [ ]:
def parse_args(**kwargs):
    parser = argparse.ArgumentParser(description='Luna nodule detection by pyTorch')
    parser.add_argument("--cfg", type=str, default="configs\\config03_win.yaml", 
                        help="Configuration")
    return parser.parse_args(**kwargs)

In [ ]:
def makeNotExistDir(dirPath):
    if not exists(dirPath):
        pathlib.Path(dirPath).mkdir(parents=True, exist_ok=True) 
    return exists(dirPath)

In [ ]:
def getFreeId():
    
    nvmlInit()
    def getFreeRatio(id):
        handle = nvmlDeviceGetHandleByIndex(id)
        use = nvmlDeviceGetUtilizationRates(handle)
        ratio = 0.5*(float(use.gpu+float(use.memory)))
        return ratio

    deviceCount = nvmlDeviceGetCount()
    available = []
    for i in range(deviceCount):
        if getFreeRatio(i)<70:
            available.append(i)
    gpus = ''
    for g in available:
        gpus = gpus+str(g)+','
    gpus = gpus[:-1]
    return gpus

In [ ]:
def setgpu(gpuinput):
    freeids = getFreeId()
    if gpuinput=='all':
        gpus = freeids
    else:
        gpus = gpuinput
        if any([g not in freeids for g in gpus.split(',')]):
            raise ValueError('gpu'+g+'is being used')
    os.environ['CUDA_VISIBLE_DEVICES'] = gpus
    return gpus

In [ ]:
class SplitComb():
    def __init__(self, side_len, max_stride, stride, margin, pad_value):
        self.side_len = side_len
        self.max_stride = max_stride
        self.stride = stride
        self.margin = margin
        self.pad_value = pad_value
        
    def split(self, data, side_len = None, max_stride = None, margin = None):
        if side_len==None:
            side_len = self.side_len
        if max_stride == None:
            max_stride = self.max_stride
        if margin == None:
            margin = self.margin
        
        assert(side_len > margin)
        assert(side_len % max_stride == 0)
        assert(margin % max_stride == 0)

        splits = []
        _, z, h, w = data.shape

        nz = int(np.ceil(float(z) / side_len))
        nh = int(np.ceil(float(h) / side_len))
        nw = int(np.ceil(float(w) / side_len))
        
        nzhw = [nz,nh,nw]
        self.nzhw = nzhw
        
        pad = [ [0, 0],
                [margin, nz * side_len - z + margin],
                [margin, nh * side_len - h + margin],
                [margin, nw * side_len - w + margin]]
        data = np.pad(data, pad, 'edge')

        for iz in range(nz):
            for ih in range(nh):
                for iw in range(nw):
                    sz = iz * side_len
                    ez = (iz + 1) * side_len + 2 * margin
                    sh = ih * side_len
                    eh = (ih + 1) * side_len + 2 * margin
                    sw = iw * side_len
                    ew = (iw + 1) * side_len + 2 * margin

                    split = data[np.newaxis, :, sz:ez, sh:eh, sw:ew]
                    splits.append(split)

        splits = np.concatenate(splits, 0)
        return splits,nzhw

    def combine(self, output, nzhw = None, side_len=None, stride=None, margin=None):
        
        if side_len==None:
            side_len = self.side_len
        if stride == None:
            stride = self.stride
        if margin == None:
            margin = self.margin
        if nzhw is None:
            nz = self.nz
            nh = self.nh
            nw = self.nw
        else:
            nz,nh,nw = nzhw
        assert(side_len % stride == 0)
        assert(margin % stride == 0)
        side_len = int(side_len/stride)
        margin = int(margin/stride)

        splits = []
        for i in range(len(output)):
            splits.append(output[i])

        output = -1000000 * np.ones((
            nz * side_len,
            nh * side_len,
            nw * side_len,
            splits[0].shape[3],
            splits[0].shape[4]), np.float32)

        idx = 0
        for iz in range(nz):
            for ih in range(nh):
                for iw in range(nw):
                    sz = iz * side_len
                    ez = (iz + 1) * side_len
                    sh = ih * side_len
                    eh = (ih + 1) * side_len
                    sw = iw * side_len
                    ew = (iw + 1) * side_len

                    split = splits[idx][margin:margin + side_len, margin:margin + side_len, margin:margin + side_len]
                    output[sz:ez, sh:eh, sw:ew] = split
                    idx += 1

        return output

In [ ]:
class LungNodule3Ddetector(Dataset):
    
    def __init__(self, data_dir, split_path, config, phase = 'train',split_comber=None):
        assert(phase == 'train' or phase == 'val' or phase == 'test')
        self.phase = phase
        
        self.max_stride = config['max_stride']
        self.stride = config['stride']       
        
        sizelim  = config['sizelim'] /config['reso']
        sizelim2 = config['sizelim2']/config['reso']
        sizelim3 = config['sizelim3']/config['reso']
        
        self.blacklist = config['blacklist']
        self.isScale = config['aug_scale']
        self.r_rand  = config['r_rand_crop']
        self.augtype = config['augtype']
        self.pad_value = config['pad_value']

        self.split_comber = split_comber
        #idcs = np.load(split_path)
        idcs = split_path

        if phase != 'test':
            idcs = [f for f in idcs if (f not in self.blacklist)]

        self.filenames = [os.path.join(data_dir, '%s_clean.npy' % idx) for idx in idcs]
        # display(self.filenames)
        
        labels = []
        for idx in idcs:
            l = np.load(os.path.join(data_dir, '%s_label.npy' %idx))
            if np.all(l==0):
                l=np.array([])
            labels.append(l)
        # display(labels)
            
        self.sample_bboxes = labels
        if self.phase != 'test':
            self.bboxes = []

            for i, l in enumerate(labels):
                if len(l) > 0 :
                    for t in l:
                        if t[3]>sizelim:
                            self.bboxes+=[[np.concatenate([[i],t])]]
                        if t[3]>sizelim2:
                            self.bboxes+=[[np.concatenate([[i],t])]]*2
                        if t[3]>sizelim3:
                            self.bboxes+=[[np.concatenate([[i],t])]]*4
            self.bboxes = np.concatenate(self.bboxes,axis = 0)

        self.crop = Crop(config)
        self.label_mapping = LabelMapping(config, self.phase)

    def __getitem__(self, idx,split=None):
        t = time.time()
        np.random.seed(int(str(t%1)[2:7]))#seed according to time

        isRandomImg  = False
        if self.phase !='test':
            if idx>=len(self.bboxes):
                isRandom = True
                idx = idx%len(self.bboxes)
                isRandomImg = np.random.randint(2)
            else:
                isRandom = False
        else:
            isRandom = False
        
        if self.phase != 'test':
            if not isRandomImg:
                bbox = self.bboxes[idx]
                filename = self.filenames[int(bbox[0])]
                imgs = np.load(filename)
                bboxes = self.sample_bboxes[int(bbox[0])]
                isScale = self.augtype['scale'] and (self.phase=='train')
                sample, target, bboxes, coord = self.crop(imgs, bbox[1:], bboxes,isScale,isRandom)
                if self.phase=='train' and not isRandom:
                     sample, target, bboxes, coord = augment(sample, target, bboxes, coord,
                        ifflip = self.augtype['flip'], ifrotate=self.augtype['rotate'], ifswap = self.augtype['swap'])
            else:
                randimid = np.random.randint(len(self.filenames))
                filename = self.filenames[randimid]
                imgs = np.load(filename)
                bboxes = self.sample_bboxes[randimid]
                isScale = self.augtype['scale'] and (self.phase=='train')
                sample, target, bboxes, coord = self.crop(imgs, [], bboxes, isScale=False, isRand=True)
            label = self.label_mapping(sample.shape[1:], target, bboxes)
            sample = (sample.astype(np.float32)-128)/128
            return torch.from_numpy(sample), torch.from_numpy(label), coord
        else:
            imgs = np.load(self.filenames[idx])
            bboxes = self.sample_bboxes[idx]
            nz, nh, nw = imgs.shape[1:]
            pz = int(np.ceil(float(nz) / self.stride)) * self.stride
            ph = int(np.ceil(float(nh) / self.stride)) * self.stride
            pw = int(np.ceil(float(nw) / self.stride)) * self.stride
            imgs = np.pad(
                imgs, 
                [[0,0], [0, pz-nz], [0, ph-nh], [0, pw-nw]], 
                'constant',
                constant_values = self.pad_value
            )
            
            xx, yy, zz = np.meshgrid(
                np.linspace(-0.5, 0.5, int(imgs.shape[1]/self.stride)),
                np.linspace(-0.5, 0.5, int(imgs.shape[2]/self.stride)),
                np.linspace(-0.5, 0.5, int(imgs.shape[3]/self.stride)),
                indexing ='ij'
            )
            coord = np.concatenate([xx[np.newaxis,...], yy[np.newaxis,...],zz[np.newaxis,:]],0).astype('float32')
            
            imgs, nzhw = self.split_comber.split(imgs)
            
            coord2, nzhw2 = self.split_comber.split(
                coord,
                side_len = int(self.split_comber.side_len/self.stride),
                max_stride = int(self.split_comber.max_stride/self.stride),
                margin = int(self.split_comber.margin/self.stride)
            )
            
            assert np.all(nzhw==nzhw2)
            imgs = (imgs.astype(np.float32)-128)/128
            return torch.from_numpy(imgs), bboxes, torch.from_numpy(coord2), np.array(nzhw)

    def __len__(self):
        if self.phase == 'train':
            return int(len(self.bboxes)/(1-self.r_rand))
        elif self.phase =='val':
            return len(self.bboxes)
        else:
            return len(self.sample_bboxes)

In [ ]:
def augment(sample, target, bboxes, coord, ifflip=True, ifrotate=True, ifswap=True):
    #                     angle1 = np.random.rand()*180
    if ifrotate:
        validrot = False
        counter = 0
        while not validrot:
            newtarget = np.copy(target)
            angle1 = (np.random.rand() - 0.5) * 20
            size = np.array(sample.shape[2:4]).astype('float')
            rotmat = np.array([[np.cos(angle1 / 180 * np.pi), -np.sin(angle1 / 180 * np.pi)],
                               [np.sin(angle1 / 180 * np.pi), np.cos(angle1 / 180 * np.pi)]])
            newtarget[1:3] = np.dot(rotmat, target[1:3] - size / 2) + size / 2
            if np.all(newtarget[:3] > target[3]) and np.all(newtarget[:3] < np.array(sample.shape[1:4]) - newtarget[3]):
                validrot = True
                target = newtarget
                sample = rotate(sample, angle1, axes=(2, 3), reshape=False)
                coord = rotate(coord, angle1, axes=(2, 3), reshape=False)
                for box in bboxes:
                    box[1:3] = np.dot(rotmat, box[1:3] - size / 2) + size / 2
            else:
                counter += 1
                if counter == 3:
                    break
    if ifswap:
        if sample.shape[1] == sample.shape[2] and sample.shape[1] == sample.shape[3]:
            axisorder = np.random.permutation(3)
            sample = np.transpose(sample, np.concatenate([[0], axisorder + 1]))
            coord = np.transpose(coord, np.concatenate([[0], axisorder + 1]))
            target[:3] = target[:3][axisorder]
            bboxes[:, :3] = bboxes[:, :3][:, axisorder]

    if ifflip:
        #         flipid = np.array([np.random.randint(2),np.random.randint(2),np.random.randint(2)])*2-1
        flipid = np.array([1, np.random.randint(2), np.random.randint(2)]) * 2 - 1
        sample = np.ascontiguousarray(sample[:, ::flipid[0], ::flipid[1], ::flipid[2]])
        coord = np.ascontiguousarray(coord[:, ::flipid[0], ::flipid[1], ::flipid[2]])
        for ax in range(3):
            if flipid[ax] == -1:
                target[ax] = np.array(sample.shape[ax + 1]) - target[ax]
                bboxes[:, ax] = np.array(sample.shape[ax + 1]) - bboxes[:, ax]
    return sample, target, bboxes, coord

In [ ]:
class Crop(object):
    def __init__(self, config):
        self.crop_size = config['crop_size']
        self.bound_size = config['bound_size']
        self.stride = config['stride']
        self.pad_value = config['pad_value']

    def __call__(self, imgs, target, bboxes, isScale=False, isRand=False):
        if isScale:
            radiusLim = [8., 100.]
            scaleLim = [0.75, 1.25]
            scaleRange = [np.min([np.max([(radiusLim[0] / target[3]), scaleLim[0]]), 1])
                , np.max([np.min([(radiusLim[1] / target[3]), scaleLim[1]]), 1])]
            scale = np.random.rand() * (scaleRange[1] - scaleRange[0]) + scaleRange[0]
            crop_size = (np.array(self.crop_size).astype('float') / scale).astype('int')
        else:
            crop_size = self.crop_size
        bound_size = self.bound_size
        target = np.copy(target)
        bboxes = np.copy(bboxes)

        start = []
        for i in range(3):
            if not isRand:
                r = target[3] / 2
                s = np.floor(target[i] - r) + 1 - bound_size
                e = np.ceil(target[i] + r) + 1 + bound_size - crop_size[i]
            else:
                s = np.max([imgs.shape[i + 1] - crop_size[i] / 2, imgs.shape[i + 1] / 2 + bound_size])
                e = np.min([crop_size[i] / 2, imgs.shape[i + 1] / 2 - bound_size])
                target = np.array([np.nan, np.nan, np.nan, np.nan])
            if s > e:
                start.append(int(np.random.randint(e, s)))  # !
            else:
                start.append(int(target[i] - crop_size[i] / 2 + np.random.randint(-bound_size / 2, bound_size / 2)))

        normstart = np.array(start).astype('float32') / np.array(imgs.shape[1:]) - 0.5
        normsize = np.array(crop_size).astype('float32') / np.array(imgs.shape[1:])
        xx, yy, zz = np.meshgrid(
            np.linspace(normstart[0], normstart[0] + normsize[0], int(self.crop_size[0]/self.stride)),
            np.linspace(normstart[1], normstart[1] + normsize[1], int(self.crop_size[1]/self.stride)),
            np.linspace(normstart[2], normstart[2] + normsize[2], int(self.crop_size[2]/self.stride)),
            indexing='ij')
        
        coord = np.concatenate([xx[np.newaxis, ...], yy[np.newaxis, ...], zz[np.newaxis, :]], 0).astype('float32')

        pad = []
        pad.append([0, 0])
        for i in range(3):
            leftpad = max(0, -start[i])
            rightpad = max(0, start[i] + crop_size[i] - imgs.shape[i + 1])
            pad.append([leftpad, rightpad])
        crop = imgs[:,
               max(start[0], 0):min(start[0] + crop_size[0], imgs.shape[1]),
               max(start[1], 0):min(start[1] + crop_size[1], imgs.shape[2]),
               max(start[2], 0):min(start[2] + crop_size[2], imgs.shape[3])]
        crop = np.pad(crop, pad, 'constant', constant_values=self.pad_value)
        for i in range(3):
            target[i] = target[i] - start[i]
        for i in range(len(bboxes)):
            for j in range(3):
                bboxes[i][j] = bboxes[i][j] - start[j]

        if isScale:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                crop = zoom(crop, [1, scale, scale, scale], order=1)
            newpad = self.crop_size[0] - crop.shape[1:][0]
            if newpad < 0:
                crop = crop[:, :-newpad, :-newpad, :-newpad]
            elif newpad > 0:
                pad2 = [[0, 0], [0, newpad], [0, newpad], [0, newpad]]
                crop = np.pad(crop, pad2, 'constant', constant_values=self.pad_value)
            for i in range(4):
                target[i] = target[i] * scale
            for i in range(len(bboxes)):
                for j in range(4):
                    bboxes[i][j] = bboxes[i][j] * scale
        return crop, target, bboxes, coord

In [ ]:
class LabelMapping(object):
    def __init__(self, config, phase):
        self.stride = np.array(config['stride'])
        self.num_neg = int(config['num_neg'])
        self.th_neg = config['th_neg']
        self.anchors = np.asarray(config['anchors'])
        self.phase = phase
        
        if phase == 'train':
            self.th_pos = config['th_pos_train']
        elif phase == 'val':
            self.th_pos = config['th_pos_val']

    def __call__(self, input_size, target, bboxes):
        stride = self.stride
        num_neg = self.num_neg
        th_neg = self.th_neg
        anchors = self.anchors
        th_pos = self.th_pos
        struct = generate_binary_structure(3, 1)

        output_size = []
        for i in range(3):
            assert (input_size[i] % stride == 0)
            output_size.append(int(input_size[i] / stride))

        label = np.zeros(output_size + [len(anchors), 5], np.float32)
        offset = ((stride.astype('float')) - 1) / 2
        oz = np.arange(offset, offset + stride * (output_size[0] - 1) + 1, stride)
        oh = np.arange(offset, offset + stride * (output_size[1] - 1) + 1, stride)
        ow = np.arange(offset, offset + stride * (output_size[2] - 1) + 1, stride)

        for bbox in bboxes:
            for i, anchor in enumerate(anchors):
                iz, ih, iw = select_samples(bbox, anchor, th_neg, oz, oh, ow)
                label[iz, ih, iw, i, 0] = 1
                label[:, :, :, i, 0] = binary_dilation(label[:, :, :, i, 0].astype('bool'), structure=struct,
                                                       iterations=1).astype('float32')

        label = label - 1

        if self.phase == 'train' and self.num_neg > 0:
            neg_z, neg_h, neg_w, neg_a = np.where(label[:, :, :, :, 0] == -1)
            neg_idcs = random.sample(range(len(neg_z)), min(num_neg, len(neg_z)))
            neg_z, neg_h, neg_w, neg_a = neg_z[neg_idcs], neg_h[neg_idcs], neg_w[neg_idcs], neg_a[neg_idcs]
            label[:, :, :, :, 0] = 0
            label[neg_z, neg_h, neg_w, neg_a, 0] = -1

        if np.isnan(target[0]):
            return label
        iz, ih, iw, ia = [], [], [], []
        for i, anchor in enumerate(anchors):
            iiz, iih, iiw = select_samples(target, anchor, th_pos, oz, oh, ow)
            iz.append(iiz)
            ih.append(iih)
            iw.append(iiw)
            ia.append(i * np.ones((len(iiz),), np.int64))
        iz = np.concatenate(iz, 0)
        ih = np.concatenate(ih, 0)
        iw = np.concatenate(iw, 0)
        ia = np.concatenate(ia, 0)
        flag = True
        if len(iz) == 0:
            pos = []
            for i in range(3):
                pos.append(max(0, int(np.round((target[i] - offset) / stride))))
            idx = np.argmin(np.abs(np.log(target[3] / anchors)))
            pos.append(idx)
            flag = False
        else:
            idx = random.sample(range(len(iz)), 1)[0]
            pos = [iz[idx], ih[idx], iw[idx], ia[idx]]
        dz = (target[0] - oz[pos[0]]) / anchors[pos[3]]
        dh = (target[1] - oh[pos[1]]) / anchors[pos[3]]
        dw = (target[2] - ow[pos[2]]) / anchors[pos[3]]
        dd = np.log(target[3] / anchors[pos[3]])
        label[pos[0], pos[1], pos[2], pos[3], :] = [1, dz, dh, dw, dd]
        return label

In [ ]:
def collate(batch):
    try:
        from collections.abc import Iterable
    except ModuleNotFoundError:
        from collections import Iterable
    
    if torch.is_tensor(batch[0]):
        return [b.unsqueeze(0) for b in batch]
    elif isinstance(batch[0], np.ndarray):
        return batch
    elif isinstance(batch[0], int):
        return torch.LongTensor(batch)
    # elif isinstance(batch[0], collections.Iterable):
    elif isinstance(batch[0], Iterable):
        transposed = zip(*batch)
        return [collate(samples) for samples in transposed]

In [ ]:
def test(data_loader, net, get_pbb, save_dir, config, numPerRun=1):
    
    start_time = time.time()

    save_dir = os.path.join(save_dir, 'bbox')
    if not os.path.exists(save_dir): os.makedirs(save_dir)
    
    net.eval()
    namelist = []
    split_comber = data_loader.dataset.split_comber
    
    for i_name, (data, target, coord, nzhw) in enumerate(data_loader):
        target = [np.asarray(t, np.float32) for t in target]
        lbb  = target[0]
        nzhw = nzhw[0]
        
        # name = data_loader.dataset.filenames[i_name].split('-')[0].split('/')[-1].split('_clean')[0]
        path, name = os.path.split(data_loader.dataset.filenames[i_name])
        name = name.split('_clean')[0]
        namelist.append(name)

        data = data[0][0]
        coord  = coord[0][0]
        isfeat = False
        
        if 'output_feature' in config:
            if config['output_feature']:
                isfeat = True
        print(); print(data.size())

        n_per_run = numPerRun
        splitlist = list(range(0,len(data)+1, n_per_run))
        
        if splitlist[-1] != len(data):
            splitlist.append(len(data))

        outputlist = []
        featurelist = []

        for i in range(len(splitlist)-1):
            input = Variable(data[splitlist[i]:splitlist[i+1]].cuda())
            inputcoord = Variable(coord[splitlist[i]:splitlist[i+1]].cuda())
            if isfeat:
                with torch.no_grad():
                    output,feature = net(input,inputcoord)
                featurelist.append(feature.data.cpu().numpy())
            else:
                with torch.no_grad():
                    output = net(input, inputcoord)
            outputlist.append(output.data.cpu().numpy())
            del input, inputcoord, output
            torch.cuda.empty_cache()
        
        output = np.concatenate(outputlist,0)
        output = split_comber.combine(output,nzhw=nzhw)
        
        if isfeat:
            feature = np.concatenate(featurelist,0).transpose([0,2,3,4,1])[:,:,:,:,:,np.newaxis]
            feature = split_comber.combine(feature,sidelen)[...,0]

        thresh = -3
        pbb,mask = get_pbb(output,thresh,ismask=True)

        if isfeat:
            feature_selected = feature[mask[0],mask[1],mask[2]]
            np.save(os.path.join(save_dir, name+'_feature.npy'), feature_selected)
            
        #tp,fp,fn,_ = acc(pbb,lbb,0,0.1,0.1)
        #print([len(tp),len(fp),len(fn)])
        print([i_name, name, data_loader.dataset.filenames[i_name]])

        np.save(os.path.join(save_dir, name+'_pbb.npy'), pbb)
        np.save(os.path.join(save_dir, name+'_lbb.npy'), lbb)
        
    np.save(os.path.join(save_dir, 'namelist.npy'), namelist)
    
    end_time = time.time()

    print('elapsed time is %3.2f seconds' % (end_time - start_time))
    return

In [ ]:
args = parse_args(args=[])
print(args)
    
bestLoss = 1000
torch.manual_seed(0)
torch.cuda.set_device(0)
    
path_root = RootPath
path_conf = join(path_root, args.cfg)
    
CFG = get_cfg_defaults()
CFG.merge_from_file(path_conf)
CFG.freeze()
print(); print(CFG)

In [ ]:
# Load model
model = import_module(CFG.TEST.MODEL)
config, net, loss, get_pbb = model.get_model()

path_ckpt = CFG.TEST.DIR_CKPT
file_ckpt = CFG.TEST.FILE_CKPT

if file_ckpt:
    abspath_ckpt = join(path_ckpt, file_ckpt)
    if exists(abspath_ckpt):
        print("\nLoad: '{}'".format(abspath_ckpt))
        pretrained = torch.load(abspath_ckpt)
        net.load_state_dict(pretrained['state_dict'])
    else:
        print("\n'{}' not exist.".format(abspath_ckpt))

display(net.state_dict)

In [ ]:
display(config)

In [ ]:
# GPU 
gpus = setgpu(CFG.TEST.GPU)
net  = net.cuda()
loss = loss.cuda()
cudnn.benchmark = True
net = DataParallel(net)

n_gpu = len(gpus.split(','))
print('Num GPU:', n_gpu)

In [ ]:
path_save = join(path_root, CFG.TEST.DIR_SAVE)
isExist = makeNotExistDir(path_save)
print("Save dir: '{}' exist? {}".format(path_save, ("True" if isExist else "False")))

path_data = join(path_root, CFG.TEST.DIR_DATA)
isExist = makeNotExistDir(path_data)
print("Data dir: '{}' exist? {}".format(path_data, ("True" if isExist else "False")))

In [ ]:
def getNameList_Csv(csvPathFile, dataPath):
    selected_names = pd.read_csv(csvPathFile).name.tolist()

    name_list = []
    for name in selected_names:
        name_str = f'{name:03d}'
        f_abspath = join(dataPath, name_str+'_label.npy')
        #print(f_abspath)
        if exists(f_abspath):
            name_list.append(name_str)
    return name_list

def getNameList_Dir(dataPath):
    series_name = pd.Series(glob(dataPath + "\\*label.npy")).apply(
                lambda x: x.split("_label.npy")[0].split("\\")[-1])
    return series_name.values.tolist()

if CFG.TEST.NAMELIST:
    abspath_namelist = join(CFG.TEST.DIR_CSV, CFG.TEST.NAMELIST)
    isExist = exists(abspath_namelist)
    print("load '{}' ? {}".format(abspath_namelist, isExist))
    if isExist:
        luna_test = getNameList_Csv(abspath_namelist, path_data)
    else:
        luna_test = getNameList_Dir(path_data)
else:
    luna_test = getNameList_Dir(path_data)

print('\nLuna test:', luna_test)

In [ ]:
# Start Test
margin = 32
sidelen = 144

stride = config['stride']
max_stride = config['max_stride']
pad_value = config['pad_value']

split_comber = SplitComb(
    sidelen,
    max_stride,
    stride,
    margin,
    pad_value
)

dataset = LungNodule3Ddetector(
    path_data,
    luna_test,
    config,
    phase='test',
    split_comber=split_comber
)

test_loader = DataLoader(
    dataset,
    batch_size = 1,
    shuffle = False,
    num_workers = 0,
    collate_fn = collate,
    pin_memory=False
)

len_dset = dataset.__len__()
print('Dataset len:', len_dset)

imgs, bboxes, coord2, nzhw = dataset[0]

print('\nImage:',  imgs.shape)
print('bboxes:', bboxes)
print('coord2:', coord2.shape)
print('nzhw:',   nzhw)

In [ ]:
dt_strt = datetime.now()
print('Start: {}'.format(dt_strt))
       
test(test_loader, net, get_pbb, path_save, config)

dt_stop = datetime.now()
print('\nStop : {}'.format(dt_stop))

dt_diff = dt_stop - dt_strt
print('Diff : {}'.format(dt_diff))

In [ ]:
print('done.')